In [1]:
import tensorflow as tf
import tensorrt as trt
import uff
import graphsurgeon as gs
from tf_utils.model import Model
from keras.backend import get_session
from PIL import Image
import numpy as np
import os
from glob import glob
import ctypes
import pycuda.driver as cuda
# This import causes pycuda to automatically manage CUDA context creation and cleanup.
import pycuda.autoinit

Using TensorFlow backend.


In [2]:
# Add plugin compiled library
ctypes.CDLL("./geluPluginv2/build_trt7_demo/libGeluPlugin.so")

<CDLL './geluPluginv2/build_trt7_demo/libGeluPlugin.so', handle 5066de0 at 0x7f0b741eb390>

In [3]:
# Add paths here
final_checkpoint = "./saved_model/checkpoints/saved_model-0005.h5"
frozen_filename = "./saved_model/frozen_model/model.pb"
uff_filename = "./saved_model/frozen_model/model.uff"
if not os.path.exists("saved_model/frozen_model"):
        os.makedirs("saved_model/frozen_model")

In [4]:
print(tf.__version__)
print(trt.__version__)

1.15.2
7.0.0.11


# Utility Functions
* Create a plugin node and replace the unsupported layer with TensorRT custom layer/plugin
* Load a trained model, freeze the graph and convert to UFF file
* Build TensorRT Engine

In [5]:
def create_plugin_node(dynamic_graph):
    gelu_node = gs.create_plugin_node(
        name="GeluActivation", op="CustomGeluPlugin", typeId=0)
    namespace_plugin_map = {"GeluActivation": gelu_node}
    dynamic_graph.collapse_namespaces(namespace_plugin_map)

In [6]:
def convert_to_uff(model, frozen_filename, uff_filename):
    # First freeze the graph and remove training nodes.
    # output_name is "dense_2/MatMul" for verification
    output_names = model.output.op.name
    sess = get_session()
    frozen_graph = tf.graph_util.convert_variables_to_constants(
        sess, sess.graph.as_graph_def(), [output_names])
    frozen_graph = tf.graph_util.remove_training_nodes(frozen_graph)
    # Save the model
    with open(frozen_filename, "wb") as fptr:
        fptr.write(frozen_graph.SerializeToString())

    # Transform graph using graphsurgeon to map unsupported TensorFlow
    # operations to appropriate TensorRT custom layer plugins
    dynamic_graph = gs.DynamicGraph(frozen_graph)
    create_plugin_node(dynamic_graph)

    uff_model = uff.from_tensorflow(dynamic_graph, [output_names])
    with open(uff_filename, "wb") as fptr:
        fptr.write(uff_model)

In [7]:
def build_engine(model_file, TRT_LOGGER):
    # For more information on TRT basics, refer to the introductory samples.
    with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.UffParser() as parser:
        builder.max_workspace_size = 1 << 16
        # Parse the Uff Network
        parser.register_input("input_1", (3, 150, 150))
        parser.register_output('dense_2/Softmax')
        parser.parse(model_file, network)
        # Build and return an engine.
        return builder.build_cuda_engine(network)

# Convert Model
* Convert the Model to UFF file
* Convert the UFF to TensorRT Engine File
* Save the serialized TensorRT Engine

In [8]:
# Load the trained model
cnn_model = Model(input_shape=(150, 150, 3))
cnn_model.build_model()
cnn_model.model.load_weights(final_checkpoint)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model built and compiled successfully


## Convert to UFF

In [9]:
convert_to_uff(model=cnn_model.model, frozen_filename=frozen_filename, uff_filename=uff_filename)

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 10 variables.
INFO:tensorflow:Converted 10 variables to const ops.
Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
NOTE: UFF has been tested with TensorFlow 1.14.0.
UFF Version 0.6.5
=== Automatically deduced input nodes ===
[name: "input_1"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 150
      }
      dim {
        size: 150
      }
      dim {
        size: 3
      }
    }
  }
}
]

Using output node dense_2/Softmax
Converting to UFF graph
Converting GeluActivation as custom op: CustomGeluPlugin

DEBUG: convert reshape to flatten node
DEBUG [/usr/local/lib/python3.6/dist-packages/uff/converters/tensorflow/converter.py:96] Marking ['dense

## Convert UFF to TensorRT Engine

In [10]:
engine_filename = "./saved_model/frozen_model/model.engine"
TRT_LOGGER = trt.Logger(trt.Logger.INFO)

In [11]:
engine = build_engine(uff_filename, TRT_LOGGER)
with open(engine_filename, "wb") as f:
    f.write(engine.serialize())

# Inference with TensorRT Engine
## Helper functions
* To run the inference with streaming input
* Allocate memory in the GPU

In [12]:
def do_inference(context, bindings, inputs, outputs, stream, batch_size=1):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

In [13]:
class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()


In [14]:
def allocate_buffers(engine):
    inputs = []
    outputs = []
    bindings = []
    stream = cuda.Stream()
    for binding in engine:
        size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
        dtype = trt.nptype(engine.get_binding_dtype(binding))
        # Allocate host and device buffers
        host_mem = cuda.pagelocked_empty(size, dtype)
        device_mem = cuda.mem_alloc(host_mem.nbytes)
        # Append the device buffer to device bindings.
        bindings.append(int(device_mem))
        # Append to the appropriate list.
        if engine.binding_is_input(binding):
            inputs.append(HostDeviceMem(host_mem, device_mem))
        else:
            outputs.append(HostDeviceMem(host_mem, device_mem))
    return inputs, outputs, bindings, stream


## Do Inference

In [15]:
TEST_DATA_PATH = "./test_data"
TEST_SAMPLES = 10
test_images = glob(os.path.join(TEST_DATA_PATH, "*.jpg"))[:TEST_SAMPLES]

In [16]:
# Load saved Engine File
with open(engine_filename, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
    # trt.init_libnvinfer_plugins(TRT_LOGGER)
    # Note that we have to provide the plugin factory when deserializing an engine built with an IPlugin or IPluginExt.
    engine = runtime.deserialize_cuda_engine(f.read())

In [17]:
# Do prediction
with engine.create_execution_context() as context:
    inputs, outputs, bindings, stream = allocate_buffers(engine)
    for impath in test_images:
        img = Image.open(impath)
        img = img.resize((150, 150))
        img = np.array(img, dtype=np.float32)
        img = np.transpose(img, (2, 0, 1))
        img = np.expand_dims(img, axis=0) / 255.0
        img = img.ravel()
        inputs[0].host = img

        [output] = do_inference(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)
        print(output)
        pred = np.argmax(output)
        basename = os.path.basename(impath)
        if pred:
            print("{} : Prediction -  Dog".format(basename))
        else:
            print("{} : Prediction -  Cat".format(basename))

[0.55680126 0.44319874]
cat.8.jpg : Prediction -  Cat
[0.7626318  0.23736823]
cat.7.jpg : Prediction -  Cat
[0.67725796 0.32274204]
cat.10.jpg : Prediction -  Cat
[0.21873236 0.78126764]
dog.4.jpg : Prediction -  Dog
[0.43486238 0.5651376 ]
dog.3.jpg : Prediction -  Dog
[0.86916035 0.13083966]
cat.5.jpg : Prediction -  Cat
[0.5476549 0.4523451]
cat.6.jpg : Prediction -  Cat
[0.22099385 0.7790062 ]
dog.2.jpg : Prediction -  Dog
[0.2710958 0.7289041]
dog.1.jpg : Prediction -  Dog
[0.72174346 0.2782565 ]
cat.9.jpg : Prediction -  Cat
